<div style="text-align:center;">
  <img src="https://ase-lib.org/_static/ase256.png" width="100" alt="Centered image">
</div>

# <center>Introduction to Molecular modeling and simulation using Atomic Simulation Environment (ASE)</center>

####  The Atomic Simulation Environment (ASE) __[Larsen et al., 2017]__ is a Python library for working with atoms. It provides python modules/tools for:
      1. setting up simulations, 
      2. manipulating data/models, 
      3. running and
      4. visualizing and analyzing atomistic simulations

__Larsen, A. H., et al.__ *The Atomic Simulation Environment—A Python library for working with atoms*, J. Phys.: Condens. Matter **29**, 273002 (2017).

---







---



### To install
``` bash
Linux: apt-get install python-ase

Python: pip3 install  - - upgrade  - - user ase
```

---

### A typical ASE python script will have below imports

In [ ]:
# run this cell 
# modeling 
from ase import Atoms
from ase.build import molecule, add_adsorbate
from ase.io import write, read
from ase.build import fcc110, fcc100
from ase.constraints import FixAtoms
from ase.visualize import view

# calculators/softwares
from ase.calculators.emt import EMT
from ase.calculators.cp2k import CP2K
from ase.calculators.gromacs import Gromacs
from ase.calculators.lammpsrun import LAMMPS


# geo-opt
from ase.optimize import QuasiNewton, BFGS
from ase.optimize import QuasiNewton

# MD simulation
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase.io.trajectory import Trajectory

# setup units
from ase import units

# plotting and system
import os
import numpy as np
import nglview as nv
import matplotlib.pyplot as plt

---


### 1. Example: Adsorption of Benzene on Copper

<div style="text-align:left;">
  <img src="images/benz_on_cu.png" width="800" alt="Centered image">
</div>

J. Chem. Theory Comput. 2006, 2, 4, 1093–1105

Abstract: The adsorption of benzene on the Cu(111), Ag(111), Au(111), and Cu(110) surfaces at low coverage is modeled using density-functional theory (DFT) using periodic-slab models...

Results: Favourable hollow site (between four copper atoms) and the long-bridge site (between two copper atoms)

Here they study combinations of
 - Different surface types
 - Position of Benzene on surface (interaction site)



### 2. Modeling 

ASE provides a wide range of tools to build molecules. Refer: https://ase-lib.org/ase/build/build.html

<div style="text-align:center;">
  <img src="images/build_things.png" width="600" alt="Centered image">
</div>

In [ ]:
! pwd

In [ ]:
# Create benzene molecule
benzene = molecule('C6H6')
#benzene.rotate(90, 'x')
write("benzene.xyz", benzene)
nv.show_ase(benzene)
#view(benzene, viewer="ngl")

In [ ]:
# create copper slab
slab = fcc110('Cu', size=(8, 8, 4)) 
nv.show_ase(slab)

In [ ]:
# add adsorbate 
benzene.rotate(90, 'x')
add_adsorbate(slab, benzene, position=(11,10), height=5)
slab_vth_benzene = slab 
nv.show_ase(slab_vth_benzene)
#view(slab_vth_benzene, viewer="ngl")

# In general a detailed literature study should be done to finalize the initial positons.


### Add vaccumm and set periodic boundary conditions

In [ ]:
slab_vth_benzene.center(vacuum=18.0, axis=2)
slab_vth_benzene.set_pbc([True, True, True])
# ase.constraints.FixAtoms
view(slab_vth_benzene, viewer='ngl')

### 3. Set Calculator (set software that you want to use)

In [ ]:
slab_vth_benzene.calc = EMT()  # Basic All-atom force field - Not for production runs

# calc_cp2k = CP2K(label='benzene',
#            cutoff=250,
#            basis_set='SZV-MOLOPT-GTH',
#            pseudo_potential='auto',
#            charge=0,
#            xc='PBE',
#            print_level='LOW',
#            max_scf=20)

#slab_vth_benzene.calc = calc_cp2k

![Caclulators](images/calculators.png)

### 4. Run Geometry Optimization

In [ ]:
# BFGS object is one of the minimizers in the ASE package. Below script uses BFGS to optimize the structure
opt = BFGS(slab_vth_benzene, trajectory="optimized.traj", append_trajectory=False, logfile="opt.log")
energies = []

def print_status(a=slab_vth_benzene):
    epot = a.get_potential_energy()
    ekin = a.get_kinetic_energy()
    energies.append(epot+ekin)
    print(f' Energy | Epot = {epot:.3f} eV | Ekin = {ekin:.3f} eV | Etot = {epot+ekin:.3f} eV')

# One can attach functions to modify the output
opt.attach(print_status, interval=3)
print("Running optimization...")
opt.run(fmax=0.02, steps=20)
print("Simulation complete.")

# One can run MD 
#MaxwellBoltzmannDistribution(opt, temperature_K=300)
#mddyn_eql = NVTBerendsen(structure, timestep=0.5 * units.fs, temperature_K=300, taut=10 * units.fs, trajectory='equil.traj', logfile='equil.log', append_trajectory=True)

## 6. Post-Processing 

### Energy Profile

In [ ]:
plt.figure()
plt.plot(range(len(energies)), energies, marker='o')
plt.xlabel('Optimization Step')
plt.ylabel('Energy (eV)')
plt.title('Geometry Optimization of Solvated benzene')
plt.savefig("optimization_energy.png")
plt.show()

### View geometry optimized trajectory 

In [ ]:
frames = read('optimized.traj', index=':')
opt = Trajectory('optimized.traj')[-1]
opt.get_cell()
view(frames, viewer="ngl")


## 7. Scan the surface

In [ ]:
# Create surface and molecule
base_slab = fcc110('Cu', size=(8, 8, 4), vacuum=10.0) 
benzene = molecule('C6H6')
benzene.rotate(90, 'x')

# Define a grid of adsorption positions (x, y) in Ångström
x_positions = np.linspace(base_slab.positions[:, 0].min()+1 , base_slab.positions[:, 0].max() -1, 5)
y_positions = np.linspace(base_slab.positions[:, 1].min()+1 , base_slab.positions[:, 1].max() -1, 5)   

results = []
overview_scan= []

for i, x in enumerate(x_positions):
    for j, y in enumerate(y_positions):
        # Copy a fresh slab, coz add_adsorbate manipulates the original slab
        slab = base_slab.copy()
        mol = benzene.copy()

        # Place benzene
        add_adsorbate(slab, mol, height=6.0, position=(x, y))

        slab.center(vacuum=18.0, axis=2)
        slab.set_pbc([True, True, True])
        slab.calc = EMT()
        overview_scan.append(slab)
        # Define filenames for this run
        name = f"opt_x{i}_y{j}"
        traj_file = f"{name}.traj"
        log_file = f"{name}.log"

        opt = BFGS(slab, trajectory=traj_file, logfile=log_file)
        opt.run(fmax=0.05, steps=30)
        
        energy = slab.get_potential_energy()
        results.append(((x, y), energy))
        print(f"Done position ({x:.2f}, {y:.2f}): E = {energy:.3f} eV")

# Save energies
with open("energies.txt", "w") as f:
    for (x, y), E in results:
        f.write(f"{x:8.3f} {y:8.3f} {E:12.6f}\n")

In [ ]:
view(overview_scan, viewer="ngl")

In [ ]:
# Check output data.
x0y0 = read('opt_x0_y0.traj', index=':')
view(x0y0, viewer="ngl")

In [ ]:
# Check output data.
x2y2 = read('opt_x2_y2.traj', index=':')
view(x2y2, viewer="ngl")

In [ ]:
energies = np.loadtxt("energies.txt")

x = energies[:, 0]
y = energies[:, 1]
E = energies[:, 2]

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, c=E)
plt.xlabel("X - Pos")
plt.ylabel("Y - Pos")
plt.title("E")
plt.colorbar(label="Value")
plt.grid(True)
plt.show()

### Basic Jupyter Notebook Commands

## Keyboard Shortcuts
### Edit Mode (press `Enter` to enter)
- Ctrl + Enter – Run cell
- Shift + Enter – Run cell and go to next
- Alt + Enter – Run cell and insert new cell below
- Tab – Code completion / suggestions
- Ctrl + / – Toggle comment

### Command Mode (press `Esc` to enter)
- Enter – Switch to Edit mode
- A – Insert a new cell above
- B – Insert a new cell below
- D, D – Delete current cell
- Z – Undo delete cell
- Y – Change cell to Code
- M – Change cell to Markdown
- Shift + M – Merge selected cells
- 0, 0 – Restart kernel